# RAS Commander: Single Plan Execution

This notebook demonstrates how to execute a single HEC-RAS plan using the RAS Commander library. We'll focus specifically on running a plan with a specified number of processor cores while overwriting an existing computation folder.

## Operations Covered

1. **Project Initialization**: Initialize a HEC-RAS project by specifying the project path and version
2. **Plan Overview**: Explore the available plans in the project
3. **Core Execution Configuration**: Set the number of processor cores to use during computation
4. **Destination Folder Management**: Use and overwrite computation folders 
5. **Results Verification**: Check the results paths after computation
6. **Performance Considerations**: Understand the impact of core count on performance

Let's begin by importing the necessary libraries and setting up our environment.

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [1]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [2]:
# Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import numpy as np
import pandas as pd
from IPython import display
from pathlib import Path
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil

## Setting Up Our Working Environment

Let's set up our working directory and paths to example projects. We'll also check the number of available CPU cores on this system.

In [ ]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

In [ ]:
# Define paths to example projects
examples_dir = bald_eagle_path.parent

# Define computation output paths
compute_dest_folder = examples_dir / "compute_test"

# Check system resources
cpu_count = psutil.cpu_count(logical=True)
physical_cpu_count = psutil.cpu_count(logical=False)
print(f"System has {physical_cpu_count} physical CPU cores ({cpu_count} logical cores)")
print(f"For HEC-RAS computation, it's often most efficient to use 2-8 cores")

## Understanding the RasCmdr.compute_plan Method

Before we dive into execution, let's understand the `compute_plan` method from the `RasCmdr` class, which is the core function for running HEC-RAS simulations.

### Key Parameters

- `plan_number` (str, Path): The plan number to execute or the full path to the plan file
- `dest_folder` (str, Path, optional): Destination folder for computation
- `ras_object` (RasPrj, optional): Specific RAS object to use (defaults to global `ras`)
- `clear_geompre` (bool, optional): Whether to clear geometry preprocessor files (default: False)
- `num_cores` (int, optional): Number of processor cores to use (default: None, uses plan settings)
- `overwrite_dest` (bool, optional): Whether to overwrite the destination folder if it exists (default: False)

### Returns
- `bool`: True if the execution was successful, False otherwise

### Key Concepts

1. **Destination Folder**: By default, the simulation runs in the original project folder. Specifying a destination folder creates a copy of the project in that location for execution, leaving the original project untouched.

2. **Number of Cores**: HEC-RAS can use multiple processor cores to speed up computation. The optimal number depends on the model complexity and your computer's specifications. Generally:
   - 1-2 cores: Good for small models, highest efficiency per core
   - 3-8 cores: Good balance for most models
   - >8 cores: Diminishing returns, may actually be slower due to overhead

3. **Geometry Preprocessor Files**: These files store precomputed hydraulic properties. Clearing them forces HEC-RAS to recompute these properties, which is useful after making geometry changes.

4. **Overwrite Destination**: Controls whether an existing destination folder should be overwritten. This is a safety feature to prevent accidental deletion of important results.

## Step 1: Project Initialization

Let's initialize the HEC-RAS project using the `init_ras_project()` function.

In [ ]:
# Initialize the HEC-RAS project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

## Step 2: Explore Available Plans

Let's examine the available plans in the project to understand what we're working with.

In [ ]:
# Display the available plans in the project
print("Available plans in the project:")
display.display(ras.plan_df)

# Let's check the current setting for number of cores in the plans
print("\nCurrent core settings for plans:")
for plan_num in ras.plan_df['plan_number']:
    # Check all three core parameters
    d1_cores = RasPlan.get_plan_value(plan_num, "UNET D1 Cores")
    d2_cores = RasPlan.get_plan_value(plan_num, "UNET D2 Cores") 
    ps_cores = RasPlan.get_plan_value(plan_num, "PS Cores")
    
    print(f"Plan {plan_num}'s Existing Settings:")
    print(f"  1D Cores: {d1_cores}")
    print(f"  2D Cores: {d2_cores}")
    print(f"  Pump Station Cores: {ps_cores}")

## Step 3: Create a Destination Folder Structure

Now, let's prepare a destination folder for our computation. This allows us to run simulations without modifying the original project files.

In [ ]:
# Create a destination folder path
dest_folder = examples_dir / "compute_test_cores"

# Check if the destination folder already exists
if dest_folder.exists():
    print(f"Destination folder already exists: {dest_folder}")
    print("We'll use overwrite_dest=True to replace it")
else:
    print(f"Destination folder will be created: {dest_folder}")

## Step 4: Execute a Plan with a Specified Number of Cores

Now we're ready to execute a plan with a specified number of cores, overwriting the destination folder if it exists. This is the core functionality demonstrated in Example 5 of the original script.

In [ ]:
# Select a plan and number of cores
plan_number = "01"
num_cores = 2  # Specify the number of cores to use

print(f"Executing plan {plan_number} with {num_cores} cores...")
print(f"Destination folder: {dest_folder}")

# Record the start time
start_time = time.time()

# Execute the plan with specified parameters
success = RasCmdr.compute_plan(
    plan_number,              # The plan to execute
    dest_folder=dest_folder,  # Where to run the simulation
    num_cores=num_cores,      # Number of processor cores to use
    overwrite_dest=True       # Overwrite destination folder if it exists
)

# Record the end time and calculate duration
end_time = time.time()
duration = end_time - start_time

# Report results
if success:
    print(f"✅ Plan {plan_number} executed successfully using {num_cores} cores")
    print(f"Execution time: {duration:.2f} seconds")
else:
    print(f"❌ Plan {plan_number} execution failed")
    print(f"Time elapsed: {duration:.2f} seconds")

## Step 5: Verify Results

After execution, let's verify the results by checking the results paths and examining the destination folder.

In [ ]:
# Verify that the destination folder exists and contains the expected files
if dest_folder.exists():
    print(f"Destination folder exists: {dest_folder}")
    
    # List the key files in the destination folder
    print("\nKey files in destination folder:")
    project_files = list(dest_folder.glob(f"{ras.project_name}.*"))
    for file in project_files[:10]:  # Show first 10 files
        file_size = file.stat().st_size / 1024  # Size in KB
        print(f"  {file.name}: {file_size:.1f} KB")
    
    if len(project_files) > 10:
        print(f"  ... and {len(project_files) - 10} more files")
    
    # Check for HDF result files
    print("\nHDF result files:")
    hdf_files = list(dest_folder.glob(f"*.hdf"))
    for file in hdf_files:
        file_size = file.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"  {file.name}: {file_size:.1f} MB")
else:
    print(f"Destination folder does not exist: {dest_folder}")

In [ ]:
# Check the results path using the RasPlan.get_results_path method
# First, initialize a RAS object using the destination folder
try:
    dest_ras = RasPrj()
    init_ras_project(dest_folder, "6.6", ras_object=dest_ras)
    
    # Get the results path for the plan we just executed
    results_path = RasPlan.get_results_path(plan_number, ras_object=dest_ras)
    
    if results_path:
        print(f"Results for plan {plan_number} are located at: {results_path}")
        
        # Check if the file exists and get its size
        results_file = Path(results_path)
        if results_file.exists():
            size_mb = results_file.stat().st_size / (1024 * 1024)
            print(f"Results file size: {size_mb:.2f} MB")
    else:
        print(f"No results found for plan {plan_number} in the destination folder")
except Exception as e:
    print(f"Error checking results: {e}")

## Summary of Single Plan Execution Options

The `RasCmdr.compute_plan()` method provides a flexible way to execute HEC-RAS plans with various options. Here's a summary of the key parameters we've explored:

1. **Basic Execution**: Simply provide a plan number
   ```python
   RasCmdr.compute_plan("01")
   ```

2. **Destination Folder**: Run in a separate folder to preserve the original project
   ```python
   RasCmdr.compute_plan("01", dest_folder="path/to/folder")
   ```

3. **Number of Cores**: Control the CPU resources used
   ```python
   RasCmdr.compute_plan("01", num_cores=2)
   ```

4. **Overwrite Destination**: Replace existing computation folders
   ```python
   RasCmdr.compute_plan("01", dest_folder="path/to/folder", overwrite_dest=True)
   ```

5. **Clear Geometry Preprocessor**: Force recalculation of geometric properties
   ```python
   RasCmdr.compute_plan("01", clear_geompre=True)
   ```

6. **Combined Options**: Use multiple options together
   ```python
   RasCmdr.compute_plan(
       "01",
       dest_folder="path/to/folder",
       num_cores=2,
       clear_geompre=True,
       overwrite_dest=True
   )
   ```

### Next Steps

To further enhance your HEC-RAS automation, consider exploring:

1. **Parallel Execution**: Use `RasCmdr.compute_parallel()` to run multiple plans simultaneously
2. **Test Mode**: Use `RasCmdr.compute_test_mode()` for testing purposes
3. **Pre-Processing**: Modify plans, geometries, and unsteady flows before execution
4. **Post-Processing**: Analyze results after computation
5. **Batch Processing**: Create scripts for parameter sweeps or scenario analysis

These advanced topics are covered in other examples and documentation for the RAS Commander library.